Team 5 members: Esther (Yuqiao) Chen, Trevor Johnson, Michelle Shen, Yi Zhang

# Introduction

## Overview
Air travel is a massive industry with total revenue exceeding $200 billlion each year between 2015 and 2019, and accounts for ~5% of the US's total GDP as of 2020 [1]. Unexpected delays have a large impact on the airlines, the airport, and of course, the passengers, as they can cause significant chain reactions to the downstream scheduling and organization of later flights.
Our team of data scientists is tasked by a group of airline executives to design a system that is capable of predicting flight departure delays ahead of schedule, so airlines have the opportunity to anticipate, mitigate and reduce delays. As a result, this would increase customer satisfaction and operational efficiency while reducing associated costs.
The goal of this system is to be integrated with the existing airline coordination systems to provide early warning to the airline and airport staff so they might be able to mitigate and reduce the predicted delay. Customers will explicitly not have access to these prediction results.
Therefore the airline executives will be the primary stakeholders, and the airline/airport staff will be the secondary stakeholders of this project.

Concretely, the objective of this project is to create a predictive machine learning classifier to give adequate advance notice to airline and airport staff of potential flight delays. Our goal is to predict flight departure delay (i.e. whether a flight will be delayed by more than 15 minutes from its scheduled time) from major U.S. airports, 2 hours before the scheduled departure time.
With the 2 hours of lead time, this would allow the airlines and the airport to potentially make necessary adjustments to mitigate the delay and have the flight back on-track, and at least fairly compensate the passengers. Because our goal is to advise airline executives and give them an adequate heads up of potential flight delay, we tuned our models to prioritize capturing all the situations where there is a potential flight delay. We erred on the side of overpredicting delays as will be shown through our choice to use F2 score.

## Related Work

Predicting flight delays is an active area of research. In recent years, with the rise of Machine Learning, various techniques have been applied to this problem, such as Neural Networks[2], Linear Regression [3], SVM [4], and Gradient Boosted Decision Trees [4].
Recent state-of-the-art research papers all seem to have taken a rather broad approach to data gathering. They have taken signals from multiple channels for training and prediction, including not only the flight and weather information, but real-time air traffic messages [2], airport traffic data [4], and so on.

## Performance Evaluation

### Metrics
The response variable, whether or not a flight is delayed, was imbalanced with roughly 23% of flights being classified as delayed. Because of this, using model **accuracy** alone is not the best metric to judge model performance. However, we do have a number of other metrics available [5].

The most common metric for imbalanced data is the **F-score**. With the definition of "no delay" being a _negative_ result, and "delay" being a _positive_ result, a prediction can fall into 4 categories:
- True Positive: The flight was predicted as delayed and is actually delayed
- True Negative: The flight was predicted as not delayed and is actually not delayed
- False Positive: The flight was predicted as delayed but is actually not delayed
- False Negative: The flight was predicted as not delayed but is actually delayed

We can then define **Precision**, **Recall**, and the ordinary **F1-score** as follows:
- Precision = \# True Positive / (\# True Positive + \# False Positive)
- Recall = \# True Positive / (\#True Positive + \# False Negative)
- F1-score = 2 * Precision * Recall / (Precision + Recall)

In this case, a False Positive outcome would likely cause airport and airline staff to waste resources and spend time trying to make sure the given flight is on time for departure when it would have been on time without intervention. 

However, a False Negative outcome would completely defeat the purpose of the advance notice system we are trying to create, since the purpose of our prediction algorithm is alerting airlines to delayed flights they otherwise would not have anticipated. If a false negative occurs and the predictive algorithm reports an on-time flight when the flight is delayed in reality, internal staff would be unprepared as the flight would recieve less attention for correction, leading to downstream additional connection delays, and airlines could lose trust with customers if they do not take proper measures to alert the customers to the delay.

Given these consequences, a False Negative has arguably higher cost than a False Positive, in other words, recall is more important here than precision.

Qualitatively, we assume recall is twice as important as precision, which gives rise to our final evaluation metric of **F2-score** [6]:
$$
F_2 = \frac{5 \cdot Precision \cdot Recall}{4 \cdot Precision + Recall}
$$

There are other notable metrics that work well for imbalanced data, such as measuring the area under an ROC curve or using a precision-recall curve. Analyzing these scores was helpful to think about probability thresholds. But because of our business problem, we chose to focus on F2 score as the primary objective.

### Cross Validation Evaluation
To evaluate the success criteria (i.e. F-score) of the models, we split the data into a training set and a test set so that the model was evaluated on unseen data for a fair score. Additionally, since the models require hyper-parameter tuning via cross validation, for each fold, we used a test set (a.k.a. development set) of data to find the best combination of hyper-parameters.

Due to the Flights data containing a time series component (i.e. with a natural time progression), we split the data into **training** and **test** sets by blocks for cross validation on a rolling basis.

Specifically, we split the Flights data from 2015 to 2019, into quarters (where each quarter is 3 months since the start of the year), resulting in 20 quarters of data.
We used a 5-fold rolling cross validation for hyperparameter tuning, and we reserved a separate hold-out set for final evaluation [7].

The exact data split was as follows:

|Fold |    Training Set    | Development (Test) Set | Test Set |
|-----|--------------------|----------|----------|
|1    | 2015 Q1 to 2017 Q3 | 2017 Q4  | --- |
|2    | 2015 Q2 to 2017 Q4 | 2018 Q1  | --- |
|3    | 2015 Q3 to 2018 Q1 | 2018 Q2  | --- |
|4    | 2015 Q4 to 2018 Q2 | 2018 Q3  | --- |
|5    | 2016 Q1 to 2018 Q3 | 2018 Q4  | --- |
|Final evaluation | 2015 Q1 to 2018 Q4 | --- |2019 Q1 to 2019 Q4|

# Data Acquisition and Preprocessing

## Data Overview

The primary dataset that we used was the Flights dataset. This data is a subset of the passenger flight's on-time performance data taken from the TranStats data collection available from the U.S. Department of Transportation (DOT). A data dictionary for this dataset can be found [here](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ). The dataset is in the format of parquet files, and there are total 31,746,841 records on flights departing from all major U.S. airports for the January 2015 to December 2019 timeframe. The following are some of the key fields in this dataset that pertain to flying performance: origin and destination of flights, departure and arrival time, carrier airline information, etc.

The second dataset that we had available for predicting flight delays was the Weather dataset. This dataset was pre-downloaded from the National Oceanic and Atmospheric Administration repository. A data dictionary for this dataset can be found [here](https://docs.google.com/spreadsheets/d/1v0P34NlQKrvXGCACKDeqgxpDwmj3HxaleUiTrY7VRn0/edit#gid=0). The following are some notable features found in this dataset: wind observation, sky condition, visibility observation, etc. There are 630,904,436 records in this dataset.

The third dataset we used was the Airport Information dataset from [OpenFlights.org](https://openflights.org/data.html). This dataset contains over 10,000 ariports spanning the globe. Each entry contains the following information: airport name, IATA code, timezone, longitude and latitude information, ..., etc. This dataset helped us to join our Flights table with Weather stations together.

## EDA

In order to understand the datasets and choose what features to keep and drop for our final joined dataset, we completed EDA on all three provided datasets. We plotted graphs to visualize feature distributions and identify relevant features for feature engineering.

### Flights Dataset

Based on the preliminary exploration of the attributes available in this flight dataset, we identified 6 main categories out of 109 attributes, summarized as the following: <br /><br />

1. **Time Related**: YEAR, QUARTER, MONTH, DEP_TIME, CRS_DEP_TIME, CRS_ELAPSED_TIME, ... etc.
2. **Airport/Geographincal Information**: ORIGIN_AIRPORT_ID, DEST_AIRPORT_ID, ORIGIN_CITY_NAME, ORIGIN_STATE_ABR, ...etc.
3. **Carriers Information**: OP_UNIQUE_CARRIER, OP_CARRIER_AIRLINE_ID, OP_CARRIER, OP_CARRIER_FL_NUM
4. **Delay Information**: DEP_DELAY, DEP_DELAY_GROUP, WEATHER_DELAY, SECURITY_DELAY, ...etc.
5. **Aircraft/Taxi Information**: WHEELS_OFF, WHEELS_ON, TAXI_IN, TAXI_OUT, ...etc.
6. **Trip Information**: DISTANCE, DIVERTED, CANCELLED, ...etc.

Among the Delay Information, we figured out that DEP_DELAY could be used as our outcome variable.

**EDA on Full Flight Dataset**

***Several key variables of the dataset are listed below:***

|Property|Data|
|-----------|----|
|Total flight count|63493682|
|Total Flight delayed by 15 minutes or more|11387082|
|% delayed flight|17.93%|
|% on-time flight|80.56%|
|% cancelled flight|1.54%|
|% diverted flight|0.25%|


***Class Imbalance for Departure Delays***

The total number of flights delayed by more than 15 minutes was 11387082, which is about 17.93% of the total number of flights, and the total proportion of on-time flights was 80.56%. This showed a class imbalance which could lead to a prediction bias towards the "on-time flight".

The figure below shows the DEP_DELAY_GROUP which groups delay times by 15 minute intervals. As expected, the departure delay group distribution was skewed to the right since the majority of flights were delayed less than 15 minutes.

![distribution_delays](files/shared_uploads/yuqiaochen@berkeley.edu/distribution_delays.png)

***Factors that can cause flight delay***  <br />

**1. Airline Carrier Factor**

Because different airline carriers may have different delay rates, we calculated the percentage of delays over the total flights for each airline carrier, and also plotted a graph to show the top 10 airline carriers with the highest percentage of delays.

![airline_delay](files/shared_uploads/yuqiaochen@berkeley.edu/airline_delay.png)

From the graph above, we observed that JetBlue(B6), Frontier Airlines(F9), Virgin America(VX), Southwest Airlines(WN), Spirit Airlines(NK)... etc had the highest delay rates, and the airline carriers were indeed associated with flight delays.

**2. Time-Related Factor** 

We believed that seasonality and time-related factors such as hour, day of the week, and holidays had an impact on flight delays. Therefore, we explored a few time features in relation to the flight departure delay rates and plotted histogram graphs of the distributions.


![time_related_factors_vs_delays](files/shared_uploads/yuqiaochen@berkeley.edu/time_related_factors_vs_delays.png)

We observed the following results:
- Day of Week: Monday, Thursday and Friday have more departure delays than other days of the week
- Hour of Day: Late afternoon and early morning have more departure delays than other time of the day
- Month of Year: June, July, August and December have more departure delays than other months of the year
- Quarter of Year: Second quarter of the year has more departure delays than other quarters of the year

**3. Flight Distance Factor** 

*From PHASE 1 (Using the data from the first quarter of 2015)*: We also thought that there might be a relationship between delay and flight distance, so we plotted a histogram of average delay by flight distance group which aggregates the average departure delay by distance group with 250 miles intervals. 

![distance_delays](files/shared_uploads/yuqiaochen@berkeley.edu/distance_delays.png)

We observed from the plotted graph: Longer distance flights were delayed with an average of 20-25 minutes. For shorter distance flights, the average delay time was less than 15 minutes.

*From PHASE 2 (Using full data)*: By using the full flight dataset, we plotted this graph again and observed that the average departure delay and flight distance had a nearly uniform distribution in the histogram.

![distance_graph](files/shared_uploads/yuqiaochen@berkeley.edu/distance_graph.png)


**3. Airport Factor** 

![airport_delay](files/shared_uploads/yuqiaochen@berkeley.edu/airport_delay.png)

The top 5 airports with the most number of delays are McKinney National Airport (TKI), Youngstown-Warren Regional Airport (YNG), Wendover Airport (ENV), Wilmington Airport (ILG) and Southwest Oregon Regional Airport (OTH). This did not make much sense because these are not the busiest and most heavily traveled connection hubs in the United States as far as we know. The busiest connection hub airports such as Dallas/Fort Worth (DFW), Chicago–O'Hare (ORD), Los Angeles (LAX), and New York–Kennedy (JFK) were not found in the top 10 aiports with highest percentage of delays.

### OpenFlights Dataset

```NOTES: 
IATA code is a three-letter geocode designating many airports and metropolitan areas around the world.
Flights dataset uses IATA code as ORIGIN/DEST.
```

By comparing OpenFlights dataset and the Flights dataset, we found the following missing airports:

![missing_airports_from_openflight](files/shared_uploads/yuqiaochen@berkeley.edu/missing_airports_from_openflight.png)

We manually filled in information (IATA Code, Timezone, Longitude/Latitude) for these missing airports.

```
open_flights2 = spark.createDataFrame(pd.DataFrame({
    'iata': ['XWA', 'EAR', 'TKI', 'IFP'], 
    'icao': ['KXWA', 'KEAR', 'KTKI', 'KIFP'], 
    'lat': [48.2578135, 40.7274925, 33.1775399, 35.16558], 
    'lng': [-103.7418471, -99.0122646, -96.5926444, -114.557093], 
    'tz_db_time_zone': ['America/Chicago', 'America/Chicago', 'America/Chicago', 'America/Phoenix']
}))
```

Then we were able to find the closest single weather station for each airport based on a longitude and latitude join using the weather stations.

### Weather Dataset

**Inital EDA of Weather Dataset** <br><br>
The weather dataset consists of 630904436 records with 177 variables. <br>
For the initial EDA of the weather dataset, we focused first on understanding the variable names and contents. Secondly, we focused on narrowing down the variables due to the size of the weather dataset. <br>
Using the data dictionary, we discovered that:
1. Many variables are amalgamations of several comma-separated, related features
> e.g. The record for `WND` could be `190,1,N,0015,1`, which translates to 5 `WND` subfeatures represented in one column, one subfeature for every comma-separated value:  <br> 
> `Wind direction angle: 190 angular degrees` <br> 
> `Wind direction quality code: 1` <br> 
> `Wind direction type code: N (normal)` <br> 
> `Wind direction speed rate: 0015 meters per second` <br> 
> `Wind direction speed quality code: 1 = Passed all quality control checks` <br> 

2. Some variables are different instances of the same type of observation
> e.g. `AA1` and `AA2` both represent episodes of liquid precipitation.

**Preliminarily reducing number of variables**

Because there were 177 variables in the Weather Data, a reduction of a single column of variables could significantly improve the runtime of our join. Alternatively, *not reducing* the number of variables before joining could cause our join to stagnate. As a result, due to the large number of variables we noted were missing data, we explored the percentage of each column consisting of missing values in order to discard variables with a high percentage (>50%) of missing values. In addition, our research led us to hypothesize that inclement weather could be a major cause of flight delays [8]. Therefore, we retained some variables despite their possessing a high percentage of missing values if their descriptions revealed them to be indicators that could capture precipitation, fog, or low visibility.

To do this, we completed the following steps:

1. Subsetted the weather dataset to a (pseudo)random sample of 30% of total records (189257881 records); we made the assumption that this sample's data distribution was representative of the larger dataset
2. Obtained the number of records for each variable that contained "missing" values (null, empty string, NA, or blank entries)
3. Divided the number of null values by the number of total records in the subset and converted it to a percentage
4. Filtered out variables for which 50% or less of their records were filled; kept the remaining 20 variables
5. Manually selected for some fog- and precipitation-related variables based on the initial data dictionary exploration and added them back into the group we planned to keep despite the low percentage of data that they represent

The table below shows the variables we kept when executing the join. For those variables that were instances of multiple observations of the same weather occurrence, we usually kept only the first of those observations because most were so sparsely populated that it didn't make sense to keep multiples.

|Variable Name|Percentage of Data Filled|
|-------------------|-----|
|STATION|99.38|
|SOURCE|87.01|
|DATE|100.00|
|LATITUDE|100.00|
|LONGITUDE|100.00|
|ELEVATION|100.00|
|NAME|99.24|
|REPORT_TYPE|100.00|
|CALL_SIGN|100.00|
|QUALITY_CONTROL|100.00|
|WND|100.00|
|CIG|100.00|
|VIS|100.00|
|TMP|100.00|
|DEW|100.00|
|SLP|100.00|
|GA1|50.67|
|GF1|57.27|
|MA1|78.95|
|AA1|23.32|
|AA2|3.15|
|AJ1|1.16|
|AL1|0.16|
|AN1|0.12|
|AO1|12.74|
|AU1|4.74|
|AT1|0.32|


After splitting variables, some EDA was performed to look at whether the subvariables might be impactful throughout the year based on a random sample of 10% of the Weather dataset. The following plot shows the distribution of the average amount of liquid precipitation in millimeters over the 52 weeks of the year. This distribution suggests that if liquid precipitation is impactful to flight delays, it is relatively evenly represented in the data, assuming that the 10% sample was representative of the distribution of the full Weather dataset.

![weather_1](files/shared_uploads/yuqiaochen@berkeley.edu/weather_1.png)


The following plot shows the distribution of the average amount of snow and ice on the ground in centimeters over the 52 weeks of the year. This distribution suggests that if snow and ice on the ground is impactful to flight delays, the months from late October to April might be significant. Therefore, some feature engineering was necessary.

![weather_2](files/shared_uploads/yuqiaochen@berkeley.edu/weather_2.png)

For more on EDA, please see the following notebooks:
* Phase 1-3 cumulative summary of EDA [here](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/2272311596985675/command/1038316885898617)
* Phase 1: [Flight](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1038316885899293/command/1038316885899295) and [Weather](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1038316885902791/command/1038316885902793)
* Phase 2: [Flight](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/3816960191362867/command/3816960191362868) and [Weather](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/3816960191359511/command/3816960191359512)
* Weather EDA plots [here](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102438204/command/1858507102438207)
* Data dictionary exploration: [Flight](https://docs.google.com/spreadsheets/d/1VKBW1Grj1eMToz9rRHRpO9RxljPsJ3Fs/edit?usp=sharing&ouid=114707694286470350740&rtpof=true&sd=true) and [Weather](https://docs.google.com/spreadsheets/d/1xthRtOjC5-kV0LMIRa_grkxZ6sLBlpqDpNwygOH_jlA/edit?usp=sharing)

## Joining the Datasets

We diagrammed our join process in the figure below:

This first diagram shows how we handled missing airport data and generated a mapping table containing the closest single weather station for each airport.

![Joining_Process_1](files/shared_uploads/yuqiaochen@berkeley.edu/Joining_Process_1.png)

This second diagram shows how we performed the joins to get our final dataset.

![Joining_Process_2](files/shared_uploads/yuqiaochen@berkeley.edu/Joining_Process_2.png)

We performed the following processes to generate the mapping table: 
1. Manually filled in data (ICAO, IATA, LONGITUDE, LATITUDE) for the missing airports that were not in the OpenFlights dataset. 
2. Joined the enriched OpenFlights dataset that contained all airport codes and lon/lat info with the Weather table on the closest longitude and latitude, to get the mapping table containing the closest single weather station for each airport.

We performed two final joins:
1. Joining the mapping table containing the closest single weather station for each airport with the Weather table on StationID to get an enriched Weather table with airport codes.
2. Joining the enriched Weather table with the Flights dataset on both `IATA` = `ORIGIN/DEST` and the nearest weather `Timestamp` to two hours prior to flight departure time.

To ease the computation on the time dimension, we rounded the weather measurements to top of the hour and we took 3 hours before the flight time, and also rounded to top of the hour, and performed a join on that. Essentially for each flight, we first took all the available weather measurements at both origin and destination between 2 and 3 hours before scheduled departure, then simply took the earilest of the measurements per location. Even though this introduced some variety on the time of the weather measurement, it sped up the join computation significantly, so we felt it was worth it.

A demonstration of our preliminary join exploration is documented in [this linked workbook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1038316885899254/command/1038316885899260).
Our final process to join the data sets together is performed in [this workbook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/3816960191354743/command/3816960191362643)

## Splitting the Weather Dataset

In order to give weather variables containing more than one subvariable meaning and to make them usable as features, we used the provided documentation to help generate a series of dictionaries allowing us to store information about each subvariable. A limitation of this method was the initial investment in hardcoding data into the dictionaries, but the result was modularized dictionaries and functions that could be easily updated if we wanted to change the way we typecasted our variables. After splitting, quality code subvariables describing the quality of another subvariable under the same supervariable were dropped, as measurements were generally of good quality. In addition, several categorical variables that were found to contain all null values were also dropped. Numerical variables were cast as doubles. Scaling factors in numerical variables were also accounted for so that those subfeatures with scale factors were automatically scaled back to a scale factor of 1 during the generation of new column variables. 

The following is an example of variables before and after splitting:<br><br>
_Before (supervariable):_ <br>
|'WND_WTHR'|
|-------------------|
|'190,1,N,0015,1'|

_After (subvariable):_ <br>
|WND_WTHR_direction_angle|WND_WTHR_direction_quality_code|WND_WTHR_type_code|WND_WTHR_speed_rate|WND_WTHR_speed_quality_code|
|---------------|-----------------|----------------|----------------|-------------|
|190.0|'1'|'N'|15.0|'1'|

For more on weather variable splitting functions, please see [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709969671/command/4070574709969672).

# Feature Engineering

## Dropped Variables

##### 1. Dropping duplicates

![duplicate_flights](files/shared_uploads/yuqiaochen@berkeley.edu/duplicate_flights.png)

From our EDA, we found that there were 50% duplicated records (31756841 records out of 63493682 records) in the Flights dataset, so our first step was to drop duplicates from that dataset and to keep only unique records.

##### 2. Dropping cancelled flights
![cancel_flight](files/shared_uploads/yuqiaochen@berkeley.edu/cancel_flight.png)

After checking the documentation of flight dataset, we knew that cancelled flights could result from the following reasons: Carrier, Weather, National Air System or Security. By doing EDA, we found that there were 1.5% flights that had been cancelled and there were 1.5% records had the DEP_DEL15 as NULL, which meant that these records with NULL as DEP_DEL15 were the cancelled flights. Since the percentage of cancelled flight was relatively small, we considered cancelled flights a different set of circumstances than flight delays. As a result, we designated DEP_DEL15 as our target variable and we removed the cancelled flight records from the Flights table.

##### 3. Dropping diverted flights
![divert_flight](files/shared_uploads/yuqiaochen@berkeley.edu/divert_flight.png)

![flight_table_info](files/shared_uploads/yuqiaochen@berkeley.edu/flight_table_info.png)

From the documentation of Flights dataset, we know that a diverted flight is a flight that is required to land at a destination other than the original scheduled destination for reasons beyond the control of the pilot/company. We omitted all diverted flights from our Flights table since the percentage of diverted flights over total records in the table was relatively small (0.25%) and was not relevant to our analysis.

## Duplicates and Outliers
To detect outliers, we created boxplots to explore variable distributions and more easily detect extreme values. To detect missing values, we performed group by operations to count the number of times missing values occurred. Additionally, several of the weather features noted above had values of "999" (or a similar string of one to six 9's) which meant that the particular observation was missing, and should not be treated as an outlier. We treated these "999" values the same way we treated the missing values in the dataset.

We handled missing values and outliers differently depending on the model we used:
- For our __logistic regression__ model, we had to make sure to identify missing values and extreme outliers.   
  - Categorical features: For variables with several missing values, we imputed a value called "missing" so these observations could still be used in our model.
  - Continuous features: For variables with several missing values, we performed median imputation. The median value for each observation came from the median value from the corresponding airport and month.
  - For both categorical and continuous variables with very few missing values, we removed these observations from the dataset. Some weather features had way too many missing values and had to be removed from the dataset altogether. 
  - We removed several extreme values from the training data to avoid skewing the logistic regression coefficients. Removing these extreme outliers is important for a logistic regression because these points have high leverage and will cause the coefficients to be more biased.

- For our __tree based__ random forest and XGBoost models, missing and extreme values are slightly less important. However, for simplicity we ended up starting with a training dataset similar to the one used for logistic regression with several missing values imputed and outliers removed. Only a few features remaining had missing values, for which we performed median or 0 imputation before modeling. 
  - For missing data, tree models can use "surrogate splits" which default to a different variable to control the split if a given observation is missing from the primary variable. Additinally, tree models are not very biased when dealing with extreme values due to how feature are binned. For these reasons we were less worried about missing values and outliers in these models.

Here are a few boxplots showing the distribution of some key variables that show outlier values:

Weather Dataset:

<img src="files/shared_uploads/trevorj@berkeley.edu/snow_boxplot.png" width="30%">

<br><br>

<img src="files/shared_uploads/trevorj@berkeley.edu/precipitation_boxplot.png" width="30%">





Flight Dataset:

![flight_boxplots](files/shared_uploads/yuqiaochen@berkeley.edu/flight_boxplots.png)

## Data Imputation

### Flight Datasets Imputation
To deal with nulls in the flight dataset, we have implemented an impute_null_numeric function to impute numerical missing data. It takes in a list of numeric features that we want to impute null values for and also the method that we want to use to do the imputation (it can either be median or mean), and then it groups the records by year, month and airport to do the calculation. By calling this function, we could more systematically deal with null value removal.

### Weather Dataset Imputation
For numerical variables, we used arbitrary value imputation [9] on numerical missing data, where a specific value was selected to represent missing data, as we suspected that missing weather values were missing not at random; therefore mean and median imputation would make less sense. The values we chose to impute for numerical variables can be found in [this spreadsheet](https://docs.google.com/spreadsheets/d/1xthRtOjC5-kV0LMIRa_grkxZ6sLBlpqDpNwygOH_jlA/edit#gid=2144784504). We recognize that this arbitrary value imputation can drastically change the distribution of variables. Categorical variables were encoded into individual binary variables. 

<br>**Imputation of numerical variables:** <br>
_Before:_ <br>
|WND_WTHR_speed_rate|
|-------------------|
|null|
|null|
|15.0|

_After:_ <br>
|WND_WTHR_speed_rate|
|-------------------|
|0.0|
|0.0|
|15.0|

**Categorical variable casting to binary:** <br>
_Before:_ <br>
|Other columns|WND_WTHR_type_code|
|------------------|------------------|
|Other columns|'N'|
|Other columns|'R'|
|Other columns|'Q'|

_After:_ <br>
|Other columns|WND_WTHR_type_code_N|WND_WTHR_type_code_R|WND_WTHR_type_code_Q|
|------------------|------------------|------------------|------------------|
|Other columns|1|0|0|
|Other columns|0|1|0|
|Other columns|0|0|1|

For more on weather variable imputation functions development, please see [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709969671/command/4070574709969672).

## Derived Features

##### 1. Extraction of local departure hour value from CRS_DEP_TIME
Intuitively, we felt that the departure time of flights might be in relation to the flight delays, therefore we decided to extract out the local departure hour value from CRS_DEP_TIME variable.

![hour_vs_delays_phase2](files/shared_uploads/yuqiaochen@berkeley.edu/hour_delays.png)

##### 2. Holiday Indicator

Since we noticed in EDA that there were some peaks around the holiday seasons, we included a holiday indicator, which indicates if a date is a public holiday such as Independence Day, Thanksgiving, Christmas, ...etc. We labeled the holiday itself as '1 = holiday', the days on either side of it as '2 = nearby holiday', and days that were not holidays as '0 = not a holiday', since we wanted to capture these patterns in our model.

We created a table containing all holidays we will consider from 2015 to 2019:

![holiday_table](files/shared_uploads/yuqiaochen@berkeley.edu/Holidays_2015_2019.png)

And a list of dates that were near holidays:

![near_holiday_dates](files/shared_uploads/yuqiaochen@berkeley.edu/near_holiday_dates.png)

We also plotted a graph showing the delay rates for these 3 categories we created (Not a Holiday/Holiday/Near Holiday)

![holiday_graph](files/shared_uploads/yuqiaochen@berkeley.edu/holiday_graph.png)

##### 3. Previous Flight Delay Indicator & Other Related Features

- Previous Flight Delay 15 mins or more (**Previous_Flight_Delay_15**)
- Enough Time between Estimate Arrival Time of Previous-Flight and Planned Departure Time of Current Flight (**Enough_Time_Btwn_Estimate_Arrival_and_Planned_Dep**)
- Poor Scheduling Indicator (**Poor_Schedule**)

We believed that flight delays can have a cumulative effect, meaning if there is any flight being delayed, later flights can be impacted as well, so we decided to create new features relating flights to previous flights.
We generated a variable to track if the previous flight was delayed or not, and we calculated the time between the estimated arrival time of the previous flight and the planned departure time of the current flight to see if there was enough time to prepare for the next flight. 
Besides that, we also found there were some flights for which the Scheduled Arrival Time of Previous-Flight was later than the Scheduled Departure Time of Current-Flight. For this kind of poor scheduling of flights, the delay rate was 87.42%, so we also added a poor scheduling indicator to capture these flights.


For more on derived features, please see [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324249652/command/4070574709970056).

## End-to-end Pipeline

In summary, we performed the following steps from the raw data to have it ready for modeling
1. Read in raw data
2. Selected a subset of raw columns that we thought would be useful
3. Removed duplicated, cancelled, and diverted flights
4. Identified the closest weather station to each airport and build an airport/station mapping table
5. Used the mapping table to join weather data at origin as well as destination airport onto the flight data at 2 hours before the flight's scheduled departure
6. **Check point 1**: saved the joined data to blob storage
7. Expanded the weather data into multiple columns, scale, and impute them as described above
8. Added derived features as described above
9. One-hot encoded a number of categorical features with manual defaults
10. Removed the outliers as described above
11. **Check point 2**: saved the processed data to blob storage

After check point 2, the data was ready to be modeled. The code for this process is captured in [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324238584/command/1898361324238594).

Additionally, many of our data engineering processes are captured in self-contained functions that were reused in [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324234161/command/1898361324234162)

# The Models
After feature engineering work was completed, tackled the prediction problem with Machine Learning models. In this section, we discuss the models we explored.

## Logistic Regression Models

Because predicting flight delay is a binary classification problem, we used Logistic Regression as a baseline model.

Logistic Regression essentially assumes a generalized linear relationship between the features and the classification label, where it uses a sigmoid function on top of a linear combination of features, specifically:
$$
p(x) = \frac{1}{1 + e^{-\beta x}} = \frac{e^{\beta x}}{e^{\beta x} + 1}
$$

where \\(p(x)\\) is the predicted label of a data point with feature vector \\(x\\), and \\(\beta\\) is the coefficients vector. The sigmoid function serves to map a predicted value into a value between 0 and 1.

To refine the Logistic Regression Model, we did the following:
- Added in the categorical and derived features we prepared from our feature engineering process
- Performed more thorough PCA and determined a more optimal value
- Performed more thorough hyper-parameter tuning on the model, e.g. regularization parameters

We expected this fine-tuning would help improve the performance of the Logistic Regression model, especially its recall and F-2 score, as its the primary performance metric.

## Decision Tree Models

In addition to the logistic regression model, we also built a gradient boosted decision tree using XGBoost as well as a Random Forest Classifier model. We used these three models in our final ensemble model to make the final prediction. 

Before jumping into modeling, we created variable importance plots to see how often our decision trees decided to use particular variables in the spilts. We included every single available feature into a random forest model, then observed the top 10% of most used features. The plot below shows the top 10% most used features in this model. After observing this, we were better informed as to which variables to make sure to include in our decision tree models. 

<img src="files/shared_uploads/trevorj@berkeley.edu/Tree_var_imp.png" width="35%">

<br>

We similarly did this for our XGBoost model. The results were similar, but not exactly the same. The XGBoost model gave very high importance to both the origin and destination airlines, whereas the random forest classifier gave less weight to these features. However it was clear to see that many of the most important features in the random forest model were also given high importance in XGBoost. We also found that several high-importance features were some of our derived features.

<img src="files/shared_uploads/trevorj@berkeley.edu/boosting_varimp.png" width="35%">


Because random forest and XGBoost both do implicit variable selection, it's not imperative to limit the number of features in the final model. Using the variable importance plots was helpful to make sure we included the most important features in our tree models. In the end, we decided to keep about 60 features in our final tree models to keep them parsimonious. 

Next, we performed hyperparameter tuning using 5-fold cross validation to test out different combinations of hyperparameters. 

Some hyperparameter tuning for the XGBoost model can be found in [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102378953/command/1858507102378955). After hyperparameter tuning, we found the following hyperparameters to yield a maximum F2 score for the XGBoost model:
- Max Depth: 10
- Number of Trees: 60
- Learning Rate: 0.1
- Colsample by Tree: .8
- Gamma = 0.05
- Alpha = 0
- Lambda = .1

Some hyperparameter tuning for the Random Forest model can be found in [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102383849/command/1858507102385645). The following hyperparameters yielded the maximum F2 score for the Random Forest model:
- Max Bins: 370
- Max Depth: 12
- Number of Trees: 50
- Feature Subset Strategy: 'sqrt'
- Subsampling Rate: 0.8

As explained earlier, selecting the optimal hyperparameters was time consuming given the large training dataset and limit cluster resources. Thus, we used 25% of the dataset for hyperparameter tuning, and fit 1 final model on the full training dataset. 

Using this method, the XGBoost model took roughly 30-45 minutes to evaluate one set of hyperparameters through 5-fold cross validation. XGBoost then took about 45 minutes to fit the full training dataset. We created a function to clean the raw data and make predictions with XGBoost. This function generally takes about 7 minutes to evaluate on the test dataset. 

The Random Forest model took roughly 60 minutes to evaluate one set of hyperparameters through 5-fold cross validation because I was testing out bigger and more trees. Then it took 15 minutes to fit the full training dataset. Like XGBoost, we created a specific function for data cleaning and making predictions for the random forest. This function takes roughtly 3 minutes to evaluate on the test data. 

The random forest model works by running each observation through the 50 decision trees, and using the majority predicted class as the final predicted probability. 

![RF_diagram](files/shared_uploads/trevorj@berkeley.edu/random_forest_diagram.png)

XGBoost on the other hand works by building iterative trees. It begins with the average log odds of the predicted class of the overall dataset. Then creates a single decision tree to try and predict the residuals of each observation. Then it takes those predicted residuals, multiplies each by a learning rate parameter, and adds this value to the overall average log odds prediction. Next, the model builds another decision tree to predict the updated residuals. Then those new residuals are once again multiplied by the learning rate and added to the running chain of predicted log odds probabilities. This continues until early stopping criteria is met or the maximum number of trees are built. 


>Final_Prediction = (Overall_log_odds) + learning_rate * (tree_1_residual) + ... + learning_rate * (tree_n_residual)


![xgb_diagram](files/shared_uploads/trevorj@berkeley.edu/xgb_diagram.png)

## Ensemble Model

The ensemble model takes the binary classification predictions from the logistic regression, random forest, and xgboost models and combines them to make a final prediction for the classification. We used this to take the final predicted class from each model and vote on the final predicted class. Because we have three different models making a binary classification, there are never any tied votes. 

The final predicted class in formula form can be thought of as follows:

$$
\hat{y} = round(\frac{\hat{y}_{logistic} + \hat{y}_{randomforest} + \hat{y}_{xgboost}}{3})
$$

# Experiments

We ran many experiments on different models for the flight prediciton problem. In this section, we provide details for each of our three models used in the ensemble.

## Baseline Logistic Regression Model

For the baseline Logistic regression, here is the end-to-end modeling process:
We read check-pointed data after processing and feature engineering (8 seconds). To deal with the class imbalance of the problem, we `under-sampled` the majority class (i.e. flights _not_ delayed) in the training data to match the number of delayed flights, so we have a balanced training set (40 seconds). We dropped categorical and derived features that we weren't ready to process, leaving only 354 numerical features (4 seconds). We ran a standard mean/variance based scaling on the feature sets (1.9 minutes). 

We performed a preliminary Principal Component Analysis and inspected the explained variance of the features (9.4 minutes). Based on an elbow of the individual explained variance of features, we decided to reduce the dimension of the feature space to 40. <br>
  ![explained_variance](files/shared_uploads/yizhang7210@berkeley.edu/Screen_Shot_2022_04_09_at_11_15_30_PM.png) <br>
- Performing the PCA again on the training data reducing to 40 features (9.5 minutes)
- Training the Logistic Regression model (6.4 minutes)
- Evaluate the model on the hold-out test set (i.e. flights in 2019) (3.1 minutes)

We did not perform any hyper-parameter tuning, and only used some reasonable default parameters for the Logistic Regression:
- maxIter: 15
- regParam (for regularizatoin penalty): 0.05
- elasticNetParam (for mix between L2/L1 penalty): 0.05

More details on the baseline Logistic Regression Model can be found in [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324231667/command/1898361324231668)

## Refined Logistic Regression

For a more accurate model from the baseline, we made a number of improvements to the baseline Logistic Regression Model:

We made use of the categorical features in the Flights dataset and the derived features that the baseline model ignored. This was done via MLlib's built-in string indexer and one-hot encoder (25 seconds).

For PCA, we realized that identifying the target dimension using an elbow on the size of the individual explained variance was not ideal. In the baseline model, the first 40 dimensions ended up only actually explaining ~50% of total variance. <br>
  ![cumulative_variance](files/shared_uploads/yizhang7210@berkeley.edu/Screen_Shot_2022_04_09_at_11_35_59_PM.png) <br>
  
As a result, we increased the number of target dimensions to 125, which now explains ~90% of the variance in the data (3 minutes). We utlized the cross validation we set up earlier to perform hyper-parameter tuning via grid search. We were unable to fully automate the process or perform random searching because we were not able to successfully run very long Spark jobs. However, we were able to see the result of the combinations of the following hyper-parameters (approximately 4 hours total):
  - regParam: 0.1, 0.2, 0.3
  - elasticNetParam: 0.0, 0.3, 0.5
  - maxIter: 15, 25
  
Furthermore, we also experimented with oversampling (with replacement) the minority class to make better use of the large amount of data that we had to capture more signals while still keeping the training set balanced. We tried ratios of 1.0, 1.5, and 2.0, and we did not see a visible difference in the resulting model performances (~3 hours total).

Overall we found the best hyperparameter combination of the Logistic Regression model to be:
  - maxIter: 25
  - regParam: 0.1
  - elasticNetParam: 0.0 (i.e. entirely L2)
  - over-sample ratio: 2.0 (i.e. resample the minority classes twice in the training data, and add the same number of majority class examples)

More details on the refined Logistic Regression Model can be found in [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709985129/command/1858507102396657)

## Random Forest & XGBoost Models


For the random forest classifier, we ran experiments to see which features were most important and which hyperparamters worked best. A plot of the most important features can be found in the decision tree models section, and we describe our hyperparameter tuning process in that section as well. We found that using a maximum tree depth of 12, 50 ensemble trees, and subsampling rate of 80% yielded strong performance through these experiments. 

Finally, we went through a similar process of experimentation with the XGBoost model. We explored the most important features and hyperparameters through experimentation. We found strong performance with each tree having a max depth of 10, 60 iterative trees with a learning rate of 0.1, 80% column sample by tree, gamma of .05, alpha of 0, and lambda of 0.1.

### Optimizing training time

When fitting our final models, we wanted to train our final models using the entire full dataset. However, training time was very important to our process when hyperparameter tuning through cross validation. We had limited cluster resources and time to select optimal hyperparameters. Because of these we often took smaller samples of our training dataset, then performed cross validation on these smaller dataset portions in order to try out different hyperparameter selections. We generally chose 25% of the dataset to evaluate with for on set of cross validation testing.

## Ensemble Model

For our exciting and novel approach to modeling, we built an ensemble model of 3 different models to make a final prediction. The final ensemble model consisted of a logistic regression portion that used PCA, an XGBoost model, and Random Forest Classifier. Our ensemble model takes an input dataset and makes predictions using all three models. Then, it uses the 3 predicted classifications to take a vote in order to make the final classification.

We created several functions to complete the above steps for each model. We assembled these model functions into a custom class to easily make predictions on new data. A notebook containing this custom class with ensemble prediction methods is [found here](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102412201/command/1858507102412221). 

We feel this ensemble approach will perform well on new unforseen data as it generalizes well. Training all three models took approximately 1.5 hours, and making predictions on the final 2019 data with this final ensemble pipeline took about 10 minutes.

### Optimizing training time

Training time was extremely important given the deadlines we had and the time consuming cross validation. When performing hyperparameter tuning using 5-fold cross validation, we used a smaller dataset that was about 25% of the normal training dataset size. Ideally, we'd like to experiment on the full training data, however we took this approach to optimize training time. Then when fitting the final trained models, we chose to fit them on the full train dataset in order to maximize performance. 

Finally, when we fit our final ensemble model, we did so on the full training dataset. This process takes roughtly one and a half hours to fit all three models in the ensemble. Due to the time series nature of our dataset, we'll need to refit our model often. Thus keeping the training time optimized is important so we can do so frequently.

# Results and Discussion

## Baseline Logistic Regression

The baseline logistic regression model as described above yielded the following final performance on the full datasets:

Here are the result metrics:
- F-2 score: 0.613
- Precision: 61.39%
- Recall: 61.33%
- Overall accuracy: 61.33%

With no hyper-parameter tuning, only numerical features, and a very basic logistic regression model, the overall performance of the baseline model looks respectable. We think under-sampling the majority class to have a balanced training set was a major contributor to having a fairly balanced performance here, with precision and recall both quite reasonable.

The reason why we did not use over-sampling the minority class was because one of the team members had prior experience comparing both over- and under-sampling, and under-sampling the majority class had performed better. This also had the added benefit of having a smaller overall training set, reducing the time taken to train the models.

## Refined Logistic Regression

The final evaluation of the refined logistic regression on the hold-out set including training and prediction took ~26 minutes, and here are the result metrics:
- F-2 score: 0.702
- Precision: 70.64%
- Recall: 70.31%
- Overall accuracy: 70.31%

This was a meaningful improvement to the baseline model. We think the hyper-parameter tuning played a major role here, with the `elasticNetParam` being a significant contributor. It seems L2 norm was a more appropriate metric here, hence having the penalty term to be entirely L2 made a difference. Another major contributor to the performance improvement was the derived features that were ignored in the baseline model. They seemed to have had good discriminative power.

## XGBoost

After performing the cross validation for tuning and selecting final hyperparameters described above, the final XGBoost classifier yielded the following performance on the full datasets: 

- Train F2 Score: 0.802
- Test F2 Score: 0.804

## Random Forest Classifier 

After performing the cross validation for tuning and selecting final hyperparameters described above, the final model yielded the following performance on the full datasets: 

- Train F2 Score: 0.792
- Test F2 Score: 0.799

## Ensemble

When we combined the predicted classes from all three models and used these to take a vote for the final prediction, we received the following performances: 

- Train F2 Score: 0.776
- Test F2 Score: 0.714

Individually, the XGBoost model seems to outperform the rest in terms of F2 score. However, we chose to perform an ensemble model as our novel idea to test out. We also believe that this ensemble model could generalize better to future or unforeseen data because of the various techniques it uses. It does appear however that the model might be overfitting slightly because of the F2 score of train outperforming the F2 score on test. But, the difference between the two is small.

# Algorithm Implementation

We implemented Logistic Regression from scratch, using PySpark's RDD interface directly.

We trained the model on a small slice of the Flights data (5665 data points), and tested it on 155 data points.

On this small scale example, the hand-crafted model had the following performance, around the same ball-park as the MLlib's implementation on the large data set:
- accuracy: 0.626
- precision: 0.377
- recall: 0.745
- F-1 score: 0.500
- F-2 score: 0.622

More details on the implementation can be found in [this notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363139939/command/1858507102373876)

# Limitations, Challenges and Future Work

## What Worked, What Didn't?

Hyperparameter tuning did not work as well as we would have liked. We were unable to perform the full hyperparameter tuning to the extent we wanted to given the time constraints and computing power. We often had to use smaller sampled versions of our dataset in order to actually see results in time. We also were only able to test out small ranges of potential hyperparameters. If given more time, we would have prefered to hyperparameter tune on a larger range of potential values, and do so on the full training dataset. 

Using the original response variable balance did not work well when fitting models. We tried using the training dataset as-is to fit and predict models. But, we found that when rebalancing the data to a 50-50 response variable split through downsampling, the models performed much better. Thus this was beneficial for us to discover the increased performance of downsampling. 

Fitting, saving, and loading transformers from the training dataset did not work as well as expected. When implementing on new unforseen data, the transformer objects often gave errors. We had to debug for a long time to figure out how to use these properly. However, in the end we were able to get our ensemble model to correctly make predictions on new unforseen data. 

Using PCA in the logistic regression worked well. After one-hot-encoding the feature space and including the hundreds of weather measurements, the dataset was extremely sparse and included lots of potentially correlated features. PCA was able to condense the feature space into something much more manageable and barely lost any performance with the logistic regression model. Using PCA we retained 90% of the variance and made our model more parsimonious.

## Performance

The training time of our models was highly dependent on cluster strength. As of April 10, our cluster had 6 workers that we could use to parallelize our tasks. Because we had many teams trying to fit and hyperparameter tune models over the past few weeks, the clusters may have been scaled back for some teams and full compute capacity was limited. This made it challenging for our team to hyperparameter tune over all possibilities we were interested in. We had to think more intuitively about the algorithm designs and think of the best hyperparameters that we would like to test given the time constraints. 

In addition, each of our models in ensemble accounted for different characteristics from the data, so voting might not have been the best method to select the final classification.

## Scalability

Because our data was time series based, our variables have seasonality and shifts over time. It is very important for us to keep our model trained with the latest data available. Thus, optimal training time is important for our problem. As-is, fitting all three models in our ensemble could take about an hour and a half to fit on the full training dataset. Perhaps as we collect new data, we'll want to refit our models and get rid of older data that won't be as indicative anymore. This training time of an hour and a half is not excessive, and our process should scale well as we refit with new data. Even if we had to refit our models daily, we could do so at the end of every day and the workload would be manageable.

## Gap Analysis

Looking at the leaderboard, there were not many teams that also used F-2 score as their evaluation metric. Out of those who did, our team has the highest F-2 score with our final ensemble model. However, we experimented with several different tweaks to try to improve this. One of these experiments involved tweaking the amount of undersampling done on the training data. We tried performing no undersampling and fitting on the full train dataset and compared that performance to the undersampled set. We found undersampling improved our F-2 score so we kept using this method. We also experimented with changing the elastic net parameter for the logistic regression model. For example, changing the elastic net parameter from 0.1 to 0.05 reduces the F-2 scores for the logistic regression from around 70% to around 61%. We also experimented with changing the feature set used in our decision tree models. This did not have a large impact on performance because the decision trees perform implicit variable selection. 

While we were not optimizing for F1 score when tuning our models, we did notice some groups have achieved higher F1 scores than our final ensemble model. The team with a higher F1 score than us is using a logistic regression + random forest ensemble classifier. This team is using deeper decision trees, and more iterations in their logistic regression than we are. We observed this after finalizing our model. Thus, it does give us some ideas of things we could have done to improve our model.

## Application of Course Concepts

Several concepts that we learned in class were illustrated in our code for this project:

- **One Hot Encoding**: One hot encoding is a pre-process to convert categorical variables into a form that could be provided to machine learning algorithms to improve prediction. We used logistic regression as our baseline model and it is unable to work with categorical variabls directly, so these categorical variables needed to be represented numerically in order to feed into our algorithm.

- **Standarization of Variables**: Before features are fed into training models, they need to be standardized in order for machine learning models to converge. With this project, we have applied this concept to standardize the features for our baseline logistic regression model.

- **Understanding transformations vs actions**: In class, we learned to distinguish between functions that are transformations and actions. Spark treats many functions as "transformations" which are lazily evaluated, and aren't actually performed on the full dataset until an action is triggered. It was important for us to keep this in mind when designing our code to think about how long a certain operation would take, and use proper caching to avoid re-running transformations when possible. Understanding this course concept improved developer time when we applied throughout our project. 

- **Algorithms Explored**: Throughout this class, we have learned about various of machine learning algorithms including supervised and unsupervised models. With this project we have explored Logistic Regression, Random Forest and Gradient Boosting Tree.

- **Data Checkpointing**: When deal with large amount of data as in this project, the low cost of storage v.s. the high time cost of compute becomes apparent. For a number of processes, including joining datasets, performing PCA, even logistic regression training, it could take 30 minutes or more. However, if we perform the computation once and save the results to permanent storage, reading from it usually takes less than 1 minute. Only until the last few days of the project did we realize we could have more aggressively saved more data into storage, notablyl the prediction results from various models, to help with us putting the ensemble model together.

## Future Work

For the logistic regression models, even though we achieved respectable F-2 scores, there are still a number of techniques we would like to explore, for example:
- Use different probability threshold to make delayed/not-delayed prediction,
- Experiment with more nuanced methods of deciding the final classification in our ensemble model rather than taking a vote,
- More thorough hyper-parameter searching, including some form of random search, and
- Try out having an imbalanced training set, in the hope to further increase recall and the F-2 score.

# Conclusion
 
In this project, we built a flight delay prediction classifier system. In summary, we performed the following tasks:
- Data processing and feature engineering, including adding derived time-based features
- Modeling with algorithms such as Logistic Regression, Random Forests and Gradient Boosting Trees
- Cross-validated models on a 5-fold time-rolling cross validation, and tested them on a hold-out dataset

Overall, our best model performed fairly well and were able to reach a healthy balance of precision and recall, having had a peak F-2 score of 0.804, which we hope could help our stakeholders, i.e. airline executives, to devise strategies in real time to mitigate major flight delays.

We also took some learnings from this project:
- Exploratory data analysis is very important. We derived many insights and ideas on what features and how to put various different data sets together through our EDA process
- Data scale matters. With the need to process a large amount of data, every step requires more careful thought to make sure of its correctness, and every intermediate result could be check-pointed to save time
- Ensemble model is not always better. Even with 3 fairly sophisticated and tuned models, the emsemble model still did not outperform some individual models.

# References

[1] https://www.zippia.com/advice/airline-industry-statistics/

[2] https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8903554&casa_token=Lobmd1rRmqwAAAAA:UQEoNuwul2BWBO3XOG4RLngJJtTfOHwc_X91bQcgsqyTxFTWhxa5vBiFj0WI95crqKsnoLc5ZUE&tag=1

[3] https://iopscience.iop.org/article/10.1088/1755-1315/81/1/012198/pdf

[4] https://www.sciencedirect.com/science/article/pii/S092523122101612X?casa_token=vcJPeqpJSxEAAAAA:Lks4HO1Xg2xfM7-rAMN1gj9kLaC1XBbA0X96IS7AjH8JCrYIrrNCD_Lcih9XKkrP5UO0SxtpnP0

[5] https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/

[6] https://deepai.org/machine-learning-glossary-and-terms/f-score

[7] https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4

[8] https://www.faa.gov/newsroom/inclement-weather-0?newsId=23074#:~:text=Inclement%20weather%2C%20including%20thunderstorms%2C%20snowstorms,70%20percent%20of%20all%20delays

[9] https://towardsdatascience.com/imputing-numerical-data-top-5-techniques-every-data-scientist-must-know-587c0f51552a